In [6]:
import pandas as pd 
import numpy as np
df = pd.read_csv("data/output/result.csv", header=None)
df.columns = ['accuracy', 'counterexamples checked', 'explanation', 'positive counterexamples', 'query', 'terminate', 'time', 'time learner', 'time verifier']
df.head()

,accuracy,counterexamples checked,explanation,positive counterexamples,query,terminate,time,time learner,time verifier
0,1.0,403.0,(not (or eggs breathes)),0.428571,breathes = 0,1.0,3.772221,0.191404,3.580459
1,1.0,256.0,(not (or eggs breathes)),0.428571,breathes = 0,1.0,2.313438,0.194700,2.118371
2,1.0,263.0,(not (or eggs breathes)),0.500000,breathes = 0,1.0,2.443796,0.156289,2.287163
3,1.0,349.0,(not (or eggs breathes)),0.400000,breathes = 0,1.0,3.023467,0.127053,2.896136
4,1.0,159.0,(or (and backbone venomous) (not (or eggs aqu...,0.466667,eggs = 0,0.0,61.009408,54.861349,0.910921


In [7]:
# take median
group_list = ['query']
df_med = df.groupby(group_list).median()
df_med.reset_index(inplace=True)
df_med.head()

,query,accuracy,counterexamples checked,positive counterexamples,terminate,time,time learner,time verifier
0,aquatic = 0,0.954545,389.5,0.428571,0.0,61.019881,37.877101,3.586408
1,backbone = 1,0.727273,220.5,0.464103,0.0,61.025941,34.832642,1.949248
2,breathes = 0,1.000000,306.0,0.428571,1.0,2.733631,0.173847,2.591650
3,eggs = 0,0.818182,268.5,0.466667,0.0,61.013182,48.991406,2.249623
4,legs <= 0.2,0.636364,363.5,0.445055,0.0,61.023240,35.185468,3.871168


In [8]:
# find representative explanation
query_explanations = {}
for key, item in df.groupby(['query'], as_index=False):
    item.reset_index(inplace=True, drop=True)
    explanation = item.iloc[item.index[(item['accuracy']-df_med['accuracy'][(df_med['query'] == key)].item()).abs().argsort()][0]]['explanation']
    query_explanations[key] = explanation

In [9]:
df_output = df_med
df_output['explanation'] = np.nan 
for key in query_explanations:
    df_output['explanation'][df_output['query'] == key] = query_explanations[key]
df_output.head()

,query,accuracy,counterexamples checked,positive counterexamples,terminate,time,time learner,time verifier,explanation
0,aquatic = 0,0.954545,389.5,0.428571,0.0,61.019881,37.877101,3.586408,(or (and backbone breathes) (not (or eggs air...
1,backbone = 1,0.727273,220.5,0.464103,0.0,61.025941,34.832642,1.949248,(or (and milk breathes) (not (or eggs breathe...
2,breathes = 0,1.000000,306.0,0.428571,1.0,2.733631,0.173847,2.591650,(not (or eggs breathes))
3,eggs = 0,0.818182,268.5,0.466667,0.0,61.013182,48.991406,2.249623,(or (and backbone venomous) (not (or eggs bre...
4,legs <= 0.2,0.636364,363.5,0.445055,0.0,61.023240,35.185468,3.871168,(and (or milk aquatic) (not (and breathes (or...
